In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.all import *
from local.tabular.core import *
from local.notebook.showdoc import show_doc

In [ ]:
#default_exp tabular.rapids

# Tabular with rapids

> Basic functions to preprocess tabular data before assembling it in a `DataBunch` on the GPU.

In [ ]:
#export
try: import cudf,nvcategory
except: print("This requires rapids, see https://rapids.ai/ for installation details")

## TabularProcessors

In [ ]:
#export
class CategorifyGPU(TabularProc):
    "Transform the categorical variables to that type."
    order = 1
    def setup(self, to):
        self.items = {}
        for n in to.cat_names: 
            col = to.loc[to.splits[0],n]
            if col.dtype != "object": col = col.astype("str")
            self.items[n] = nvcategory.from_strings(col.data).keys()
        to.classes = self.items
    
    def _apply_cats(self, c):
        if c.dtype != "object": c = c.astype("str")
        return cudf.Series(nvcategory.from_strings(c.data).set_keys(self.items[c.name]).values()).add(1)
    
    def encodes(self, to): 
        for c in to.cat_names: to.set_col(c, self._apply_cats(to.items[c]))
            
    #def _decode_cats(self, c): return c.map(dict(enumerate(self[c.name].items)))
    #def decodes(self, to): to.transform(to.all_cat_names, self._decode_cats)

In [ ]:
show_doc(CategorifyGPU, title_level=3)

<h3 id="CategorifyGPU" class="doc_header"><code>class</code> <code>CategorifyGPU</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>CategorifyGPU</code>(**`enc`**=*`None`*, **`dec`**=*`None`*, **`filt`**=*`None`*, **`as_item`**=*`False`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Transform the categorical variables to that type.

In [ ]:
cat = CategorifyGPU()
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,0,2]}))
to = Tabular(df, 'a')

cat.setup(to)
test_eq(list(cat.items['a'].to_host()), ['0','1','2'])
cat(to)
test_eq(df['a'].to_array(), np.array([1,2,3,1,3]))
df1 = cudf.from_pandas(pd.DataFrame({'a':[1,0,3,-1,2]}))
to1 = Tabular(df1, 'a')
cat(to1)
#Values that weren't in the training df are sent to -1 (na)
test_eq(df1['a'].to_array(), np.array([2,1,0,0,3]))

In [ ]:
cat = CategorifyGPU()
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,3,2]}))
to = Tabular(df, 'a', splits=[range(3)])
cat.setup(to)
test_eq(list(cat.items['a'].to_host()), ["0","1","2"])
cat(to)
test_eq(df['a'].to_array(), np.array([1,2,3,0,3]))

In [ ]:
#TODO Categorical

#df = pd.DataFrame({'a':pd.Categorical(['M','H','L','M'], categories=['H','M','L'], ordered=True)})
#to = Tabular(df, 'a')
#cat = Categorify()
#cat.setup(to)
#test_eq(cat['a'], ['#na#','H','M','L'])
#cat(to)
#test_eq(df.a, [2,1,3,2])
#to2 = cat.decode(to)
#test_eq(to2.a, ['M','H','L','M'])

In [ ]:
#export
class NormalizeGPU(TabularProc):
    "Normalize the continuous variables."
    order = 2
    def setup(self, to):
        df = to.loc[to.splits[0], to.cont_names]
        self.means,self.stds = df.mean(),df.std(ddof=0)+1e-7

    def encodes(self, to): to.conts = (to.conts-self.means) / self.stds
    def decodes(self, to): to.conts = (to.conts*self.stds ) + self.means

In [ ]:
#export
class NormalizeGPU(TabularProc):
    "Normalize the continuous variables."
    order = 2
    def setup(self, to):
        self.means,self.stds = {},{}
        for n in to.cont_names:
            col = df.loc[to.splits[0],n]
            self.means[n],self.stds[n] = col.mean(),col.std(ddof=0)+1e-7
    
    def encodes(self, to):
        for n in to.cont_names: to.set_col(n, (to.items[n]-self.means[n])/self.stds[n])
            
    def decodes(self, to):
        for n in to.cont_names: to.set_col(n, (to.items[n]*self.stds[n])+self.means[n])

In [ ]:
show_doc(NormalizeGPU, title_level=3)

<h3 id="NormalizeGPU" class="doc_header"><code>class</code> <code>NormalizeGPU</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>NormalizeGPU</code>(**`enc`**=*`None`*, **`dec`**=*`None`*, **`filt`**=*`None`*, **`as_item`**=*`False`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Normalize the continuous variables.

In [ ]:
norm = NormalizeGPU()
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,3,4]}))
to = Tabular(df, cont_names='a')
norm.setup(to)
x = np.array([0,1,2,3,4])
m,s = x.mean(),x.std()
test_eq(norm.means['a'], m)
test_close(norm.stds['a'], s)
norm(to)
test_close(df['a'].to_array(), (x-m)/s)
df1 = cudf.from_pandas(pd.DataFrame({'a':[5,6,7]}))
to1 = Tabular(df1, cont_names='a')
norm(to1)
test_close(df1['a'].to_array(), (np.array([5,6,7])-m)/s)
to2 = norm.decode(to1)
test_close(to2.items['a'].to_array(), [5,6,7])

In [ ]:
norm = NormalizeGPU()
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,3,4]}))
to = Tabular(df, cont_names='a', splits=[range(3)])
norm.setup(to)
x = np.array([0,1,2])
m,s = x.mean(),x.std()
test_eq(norm.means, {'a': m})
test_close(norm.stds['a'], s)
norm(to)
test_close(df['a'].to_array(), (np.array([0,1,2,3,4])-m)/s)

In [ ]:
#export
def get_median(col):
    "Get the median of a cudf Series `col`"
    col = col.dropna().reset_index(drop=True)
    return col.sort_values()[len(col)//2]

In [ ]:
#export
class FillStrategyGPU:
    "Namespace containing the various filling strategies."
    def median  (c,fill): return get_median(c)
    def constant(c,fill): return fill
    def mode    (c,fill): return c.dropna().value_counts().index[0]

In [ ]:
#export
class FillMissingGPU(TabularProc):
    "Fill the missing values in continuous columns."
    def __init__(self, fill_strategy=FillStrategyGPU.median, add_col=True, fill_vals=None):
        if fill_vals is None: fill_vals = defaultdict(int)
        store_attr(self, 'fill_strategy,add_col,fill_vals')

    def setup(self, to):
        self.na_dict = {}
        for n in to.cont_names:
            col = to.loc[to.splits[0], to.cont_names]
            if col.isnull().any(): self.na_dict[n] = self.fill_strategy(col, self.fill_vals[n])

    def encodes(self, to):
        for n in to.cont_names:
            if n in self.na_dict:
                if self.add_col:
                    to.items[n+'_na'] = to.items[n].isnull()
                    if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')
                to.set_col(n, to.items[n].fillna(self.na_dict[n]))
            elif df[n].isnull().any():
                raise Exception(f"nan values in `{n}` but not in setup training set")

In [ ]:
show_doc(FillMissingGPU, title_level=3)

<h3 id="FillMissingGPU" class="doc_header"><code>class</code> <code>FillMissingGPU</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>FillMissingGPU</code>(**`fill_strategy`**=*`'median'`*, **`add_col`**=*`True`*, **`fill_vals`**=*`None`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Fill the missing values in continuous columns.

In [ ]:
fill1,fill2,fill3 = (FillMissingGPU(fill_strategy=s) 
                     for s in [FillStrategyGPU.median, FillStrategyGPU.constant, FillStrategyGPU.mode])
df = cudf.from_pandas(pd.DataFrame({'a':[0,1,np.nan,1,2,3,4]}))
df1 = df.copy(); df2 = df.copy()
to,to1,to2 = Tabular(df, cont_names='a'),Tabular(df1, cont_names='a'),Tabular(df2, cont_names='a')
fill1.setup(to); fill2.setup(to1); fill3.setup(to2)
test_eq(fill1.na_dict, {'a': 2.})
test_eq(fill2.na_dict, {'a': 0})
test_eq(fill3.na_dict, {'a': 1.0})

fill1(to); fill2(to1); fill3(to2)
for t in [to, to1, to2]: test_eq(t.cat_names, ['a_na'])

for to_,v in zip([to, to1, to2], [2., 0., 1.]):
    test_eq(to_.items['a'].to_array(), np.array([0, 1, v, 1, 2, 3, 4]))
    test_eq(to_.items['a_na'].to_array(), np.array([0, 0, 1, 0, 0, 0, 0]))
    
dfa = cudf.from_pandas(pd.DataFrame({'a':[np.nan,0,np.nan]}))
dfa1 = dfa.copy(); dfa2 = dfa.copy()
to,to1,to2 = Tabular(dfa, cont_names='a'),Tabular(dfa1, cont_names='a'),Tabular(dfa2, cont_names='a')
fill1(to); fill2(to1); fill3(to2)
for to_,v in zip([to, to1, to2], [2., 0., 1.]):
    test_eq(to_.items['a'].to_array(), np.array([v, 0, v]))
    test_eq(to_.items['a_na'].to_array(), np.array([1, 0, 1]))

## TabularProcessor -

In [ ]:
#export
class TabularPreprocessorGPU():
    "An object that will preprocess dataframes using `procs`"
    def __init__(self, procs, cat_names=None, cont_names=None, cat_y=None, inplace=True):
        self.cat_names,self.cont_names,self.cat_y,self.inplace = L(cat_names),L(cont_names),L(cat_y),inplace
        self.procs = L(p if isinstance(p, type) else partial(TabularProc, func=p) for p in procs).sorted(key='order')
    
    def __call__(self, df, trn_idx=None):
        "Call each of `self.procs` on `df`, setup on `df[trn_idx]` if not None"
        df = df if self.inplace else df.copy()
        if trn_idx is None:
            for p in self.procs: p(df)
        else:
            self.procs,procs = [],self.procs
            for p in procs: 
                p_ = p(cat_names=self.cat_names + self.cat_y if p==CategorifyGPU else self.cat_names, cont_names=self.cont_names)
                p_.setup(df, trn_idx=trn_idx)
                p_(df)
                if p!= CategorifyGPU: self.cat_names,self.cont_names = p_.cat_names,p_.cont_names
                else:
                    self.classes = {n:'#na#'+L(p_.categories[n].to_host(), use_list=True) for n in self.cat_names + self.cat_y}
                self.procs.append(p_)
            for p in self.procs:
                if isinstance(p, Normalize): self.means,self.stds = p.means,p.stds
        return df

In [ ]:
procs = [NormalizeGPU, CategorifyGPU, FillMissingGPU, noop]
proc = TabularPreprocessorGPU(procs, 'a', 'b', inplace=False)

#Test reordering and partialize
test_eq(proc.procs, [FillMissingGPU, proc.procs[1], CategorifyGPU, NormalizeGPU])
test_eq(proc.procs[1].func, TabularProc)
test_eq(proc.procs[1].keywords, {'func': noop})

df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4]}))

#Test setup and apply on df_trn
df1 = proc(df, trn_idx=range_of(df))
test_eq(df1['a'].to_array(), [0,1,2,1,1,2,0])
test_eq(df1['b_na'].to_array(), [0,0,1,0,0,0,0])
x = np.array([0,1,2,1,2,3,4])
m,s = x.mean(),x.std()
test_close(df1['b'].to_array(), (x-m)/s)
test_eq(proc.classes, {'a': ['#na#','0','1','2'], 'b_na': ['#na#','False','True']})

#Test apply on df_val
df = cudf.from_pandas(pd.DataFrame({'a':[2,1,3], 'b':[4,5,np.nan]}))
df1 = proc(df)
test_eq(proc.classes, {'a': ['#na#','0','1','2'], 'b_na': ['#na#','False','True']})
test_eq(df1['a'].to_array(), [2,1,-1])
test_eq(df1['b_na'].to_array(), [0,0,1])
x = np.array([4, 5, 2])
test_close(df1['b'].to_array(), (x-m)/s)

#Test apply on cat_y
procs = [NormalizeGPU, CategorifyGPU, FillMissingGPU, noop]
proc = TabularPreprocessorGPU(procs, 'a', 'b', cat_y='c', inplace=False)

df = cudf.from_pandas(pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']}))
df1 = proc(df, trn_idx=range_of(df))
test_eq(proc.cat_names, ['a', 'b_na'])
test_eq(df1['a'].to_array(), [0,1,2,1,1,2,0])
test_eq(df1['b_na'].to_array(), [0,0,1,0,0,0,0])
test_eq(df1['c'].to_array(), [1,0,1,0,0,1,0])
x = np.array([0,1,2,1,2,3,4])
m,s = x.mean(),x.std()
test_close(df1['b'].to_array(), (x-m)/s)
test_eq(proc.classes, {'a': ['#na#','0','1','2'], 'b_na': ['#na#','False','True'], 'c': ['#na#','a','b']})

In [ ]:
#export
def process_df_gpu(df, splits, procs, cat_names=None, cont_names=None, cat_y=None, inplace=True):
    "Process `df` with `procs` and returns the processed dataframe and the `TabularProcessorGPU` associated"
    proc = TabularPreprocessorGPU(procs, cat_names, cont_names, cat_y, inplace=inplace)
    res = proc(df, trn_idx=splits[0])
    return res,proc

Pass the same `splits` as you will use for splitting the data, so that the setup is only done on the training set. `cat_names` are the names of the categorical variables, `cont_names` the continous ones, `cat_y` are the names of the dependent variables that are categories. If `inplace=True`, processing is applied inplace, otherwis it creates a copy of `df`.

In [ ]:
#export
class TabularLine(pd.Series):
    "A line of a dataframe that knows how to show itself"
    def show(self, ctx=None, **kwargs):
        if ctx is None: return self
        else: return ctx.append(self)

In [ ]:
#export
class TensorTabular(TupleBase):
    
    def get_ctxs(self, max_n=10, **kwargs):
        n_samples = min(self[0].shape[0], max_n)
        df = pd.DataFrame(index = range(n_samples))
        return [df.iloc[i] for i in range(n_samples)]
    
    def display(self, ctxs): display_df(pd.DataFrame(ctxs))

In [ ]:
#export
class ReadTabLine(ItemTransform):
    def __init__(self, proc): 
        self.proc = proc
        self.o2is = {n: defaultdict(int, {v:i for i,v in enumerate(proc.classes[n])}) for n in proc.cat_names}
    
    def encodes(self, row): 
        cats = [self.o2is[n][row[n]] for n in self.proc.cat_names]
        conts = [row[n] for n in self.proc.cont_names]
        return TensorTabular((tensor(cats).long(),tensor(conts).float()))
    
    def decodes(self, o):
        dic = {c: self.proc.classes[c][v] for v,c in zip(o[0], self.proc.cat_names)}
        ms = getattr(self.proc, 'means', {c:0 for c in self.proc.cont_names})
        ss = getattr(self.proc, 'stds',  {c:1 for c in self.proc.cont_names})
        dic.update({c: (v*ss[c] + ms[c]).item() for v,c in zip(o[1], self.proc.cont_names)})
        return TabularLine(pd.Series(dic))

In [ ]:
#export
class ReadTabTarget(ItemTransform):
    def __init__(self, proc): 
        self.proc = proc
        assert len(proc.cat_y) == 1
        self.o2i = defaultdict(int, {v:i for i,v in enumerate(proc.classes[proc.cat_y[0]])})
    
    def encodes(self, row): return self.o2i[row[self.proc.cat_y[0]]]-1
    def decodes(self, o) :  return Category(self.proc.classes[self.proc.cat_y[0]][o+1])

In [ ]:
tds = TfmdDS(df1, tfms=[[ReadTabLine(proc)], ReadTabTarget(proc)], use_list=None)

In [ ]:
enc = tds[1]
test_eq(enc[0][0], tensor([2,1]))
test_close(enc[0][1], tensor([-0.628828]))
test_eq(enc[1], 0)

dec = tds.decode(enc)
assert isinstance(dec[0], TabularLine)
test_close(dec[0], pd.Series({'a': 1, 'b_na': False, 'b': 1}))
test_eq(dec[1], 'a')

test_stdout(lambda: print(tds.show_at(1)), """a               1
b_na        False
b               1
category        a
dtype: object""")

## Integration example

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]

In [ ]:
splits = RandomSplitter()(range_of(df))
df1,proc = process_df(df, splits, procs=procs, cat_names=cat_names, cont_names=cont_names, cat_y="salary", inplace=False)

In [ ]:
dsrc = DataSource(df1, filts=splits, tfms=[[ReadTabLine(proc)], [ReadTabTarget(proc)]])

In [ ]:
dbch = dsrc.databunch(bs=64)
dbch.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,category
0,Private,Assoc-voc,Married-civ-spouse,Craft-repair,Husband,White,False,51.0,99063.992188,11.0,<50k
1,Local-gov,Assoc-voc,Divorced,Tech-support,Unmarried,Asian-Pac-Islander,False,44.0,73199.000000,11.0,<50k
2,Private,Assoc-voc,Married-civ-spouse,Craft-repair,Husband,White,False,39.0,53569.000000,11.0,<50k
3,Private,Some-college,Married-civ-spouse,Adm-clerical,Husband,White,False,49.0,280525.000000,10.0,>=50k
4,Private,Some-college,Never-married,Tech-support,Own-child,White,False,25.0,245628.000000,10.0,<50k
5,State-gov,Doctorate,Married-civ-spouse,Prof-specialty,Husband,White,False,41.0,116520.000000,16.0,>=50k
6,Private,HS-grad,Married-civ-spouse,Sales,Husband,Other,False,25.0,195201.000000,9.0,<50k
7,Private,HS-grad,Divorced,Craft-repair,Not-in-family,White,False,19.0,517036.000000,9.0,<50k
8,Private,Masters,Married-civ-spouse,Prof-specialty,Husband,White,False,48.0,47343.000000,14.0,>=50k
9,Self-emp-not-inc,Masters,Married-civ-spouse,Sales,Wife,White,False,41.0,186909.000000,14.0,>=50k


## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_dataloader.ipynb.
Converted 01a_script.ipynb.
Converted 02_transforms.ipynb.
Converted 03_pipeline.ipynb.
Converted 04_data_external.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_source.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 11_layers.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_metrics.ipynb.
Converted 21_tutorial_imagenette.ipynb.
Converted 30_text_core.ipynb.
Converted 31_text_data.ipynb.
Converted 32_text_models_awdlstm.ipynb.
Converted 33_test_models_core.ipynb.
Converted 34_callback_rnn.ipynb.
Converted 35_tutorial_wikitext.ipynb.
Converted 36_text_models_qrnn.ipynb.
Converted 40_tabula